# Create a time-of-flight lookup table for BIFROST

In [ ]:
from ess.reduce import time_of_flight, nexus
import sciline
import scipp as sc
import scippnexus as snx
from scippneutron.chopper import DiskChopper

from ess.bifrost import BifrostSimulationWorkflow
from ess.bifrost.data import simulated_elastic_incoherent_with_phonon
from ess.spectroscopy.types import *
from ess.bifrost.types import *

## Load data

In [ ]:
loader = BifrostSimulationWorkflow(detector_names=['_'])
loader[Filename[SampleRun]] = simulated_elastic_incoherent_with_phonon()

In [ ]:
data = loader.compute([
    Position[snx.NXsource, SampleRun],
    Position[snx.NXsample, SampleRun],
    nexus.types.Choppers[SampleRun],
    nexus.types.CalibratedMonitor[SampleRun, FrameMonitor3],
])
source_position = data[Position[snx.NXsource, SampleRun]]
sample_position = data[Position[snx.NXsample, SampleRun]]
raw_choppers = data[nexus.types.Choppers[SampleRun]]
monitor = data[nexus.types.CalibratedMonitor[SampleRun, FrameMonitor3]]

## Process choppers

In [ ]:
def parse_choppers(raw_choppers):
    choppers = sc.DataGroup()
    for name, chopper in raw_choppers.items():
        processed = chopper.copy()
        # These are constant in the simulated data.
        processed['rotation_speed'] = processed['rotation_speed'].data.mean()
        processed['phase'] = processed['phase'].data.mean()
        # Guessing here as this is not stored in the file.
        processed['beam_position'] = sc.scalar(0.0, unit='deg')
        choppers[name] = DiskChopper.from_nexus(processed)
    return choppers

In [ ]:
choppers = parse_choppers(raw_choppers)

## Setting up the workflow

The table needs to cover the range of distances for both sample and monitor.
Here, the shifts of 0.1m are arbitrarily chosen to make sure the table covers `l1` and `l_monitor`.

In [ ]:
l_monitor = sc.norm(source_position - monitor.coords['position'])
l1 = sc.norm(source_position - sample_position)
l_min = l_monitor - sc.scalar(0.1, unit='m')
l_max = l1 + sc.scalar(0.1, unit='m')

In [ ]:
simulation_results = time_of_flight.simulate_beamline(
    choppers=choppers, neutrons=5_000_000, source_position=source_position,
)

In [ ]:
wf = sciline.Pipeline(
    time_of_flight.providers(),
    params={
        **time_of_flight.default_parameters(),
        time_of_flight.LtotalRange: (l_min, l_max),
        time_of_flight.SimulationResults: simulation_results,
        time_of_flight.DistanceResolution: sc.scalar(0.1, unit='m'),
    },
)

## Compute the table

In [ ]:
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table

In [ ]:
table.squeeze().plot()

## Save to file

In [ ]:
table.save_hdf5('BIFROST-simulation-tof-lookup-table.h5')